In [ ]:
import pandas as pd

In [ ]:
csv_data=pd.read_csv("../data-files_and_results/fewnerd-data/train.txt", sep="\t", header=None, names=["-DOCSTART-", "-X- O O"], skip_blank_lines=False)

In [ ]:
started=False
for index, row in csv_data.iterrows():
    tag=row.iloc[1]
    
    if pd.isna(csv_data.iloc[index, 1]):
        continue
        
    if tag=="O":
        started=False
    
    elif tag.startswith("art") or tag.startswith("product") or tag.startswith("event"):
        csv_data.iloc[index, 1]="O"
        started=False
        
    elif tag.startswith("building") or tag.startswith("location"):
        if started==False:
            csv_data.iloc[index, 1]="B-LOC"
            started=True
            
        else:
            csv_data.iloc[index, 1]="I-LOC"
        
    elif tag.startswith("organization"):
        if started==False:
            csv_data.iloc[index, 1]="B-ORG"
            started=True
            
        else:
            csv_data.iloc[index, 1]="I-ORG"
            
    elif tag.startswith("person"):
        if started==False:
            csv_data.iloc[index, 1]="B-PER"
            started=True
        else:
            csv_data.iloc[index, 1]="I-PER"
            
    elif tag.startswith("other"):
        if tag=="other-god":
            if started==False:
                csv_data.iloc[index, 1]="B-PER"
                started=True
            else:
                csv_data.iloc[index, 1]="I-PER"
        else:
            csv_data.iloc[index, 1]="O"
            started=False
            
    else:
        print("Weird!")

In [ ]:
csv_data.to_csv("../data-files_and_results/fewnerd-data/fewnerd-supervised-training-data.csv", sep=" ", index=False)

In [ ]:
p_data=[]
for line in open("fewnerd-supervised-training-data.csv", encoding='utf-8'):
    line=line.strip()
    p_data.append(line)

# Attempts at CONLL-U format. Abandoned

In [ ]:
processed_data=p_data[:round(len(p_data)/10)+52]

In [ ]:
nr_lines=len(processed_data)
sentence=""
words=[]
lines=[]
for i, tokenandtag in reversed(list(enumerate(processed_data))):
    print("Progress... line ", nr_lines-i, " out of ", nr_lines, " | ", str(round(100*((nr_lines-i)/nr_lines))), "%", end="\r")
    if tokenandtag == "":
        sentence=" ".join(reversed(words))
        words=[]
        processed_data=processed_data[:i+1]+[f"#", f"# text = {sentence}"]+processed_data[i+1:]
    elif i==0:
        sentence=" ".join(reversed(words))
        word, _ = tokenandtag.split("\t")
        processed_data=[f"#", f"# text = {word} {sentence}"]+processed_data[i:]
    else:
        try:
            token, tag = tokenandtag.split("\t")
            words.append(token)
        except:
            print(tokenandtag)

In [ ]:
data_w_htags=processed_data
print(data_w_htags[:40])

In [ ]:
nr_lines_2=len(data_w_htags)
sent_id=1
word_cnt=1
text=False

for i, line in enumerate(data_w_htags):
    print("Line ", i, " out of ", nr_lines_2, " | ", str(round(100*(i/nr_lines_2))), "% Progress...", end="\r")
    
    if line=="":
        continue
        
    if text==True:
        text=False
        continue
        
    if line[0]=="#":
        data_w_htags[i]=f"# sent_id = {sent_id}"
        sent_id+=1
        word_cnt=1
        text=True
        
    else:
        data_w_htags[i]=f"{word_cnt}\t"+data_w_htags[i]
        word_cnt+=1

In [ ]:
done_data=data_w_htags
print(done_data[:20])

In [ ]:
with open("fewnerd-train.conllu", "w", encoding="utf-8") as f:
    f.write("\n".join(done_data))